# Cleaning 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
%matplotlib inline
path = './data/'

import statsmodels
from statsmodels.tsa.stattools import acf, ccf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics import utils

In [67]:
AQI_data = pd.read_csv(path+'AQI.csv')
AQI_data = AQI_data.drop('Unnamed: 0', 1)

#Convert date column into a "real" date column, not a floating number
AQI_data['date'] = AQI_data['date'].astype(int).astype(str)
AQI_data['date'] = pd.to_datetime(AQI_data['date']).dt.date

#Reshuffle columns so that date, hour, type are at the front
observation_cols = list(AQI_data.columns.values[:-3])
cols = list(AQI_data.columns.values[-3:])
cols.extend(observation_cols)

AQI = AQI_data[cols]
print(len(AQI))
AQI.head()

21962


,date,hour,type,1001A,1002A,1003A,1004A,1005A,1006A,1007A,...,2706A,2707A,2708A,2709A,2710A,2711A,2835A,2842A,2845A,2846A
0,2014-05-13,0.0,AQI,73.0,32.0,84.0,67.0,75.0,83.0,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-05-13,1.0,AQI,74.0,43.0,89.0,NaN,73.0,79.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-05-13,2.0,AQI,76.0,83.0,92.0,NaN,82.0,88.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-05-13,3.0,AQI,76.0,35.0,91.0,NaN,88.0,80.0,89.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-05-13,4.0,AQI,79.0,54.0,98.0,NaN,94.0,82.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### The data is hourly, but some hours are "missing" throughout the entire dataset. 
For example, there are zero records for 16:00 on 2014-05-13 -- that hour is completely blank for all stations. 

To apply time series models effectively, it would be nice to "standardize" the dataset so that all records are equally spaced apart. For example, without 16:00 on May 13, the gap between 15:00 and the preceding record (14:00) will be larger than the gap between 15:00 and the subsequent record (17:00). This is a problem. 

To address this problem, we'll construct a formal "Times" dataframe where ALL hours between May 13, 2014 and December 31, 2016 are represented. We'll merge this "Times" dataframe with the AQI dataframe. Then, we will need to estimate missing AQI values for the empty hours (newly introduced by the "Times" df.

In [6]:
Times = pd.DataFrame(pd.date_range(datetime.date(2014, 5, 13), datetime.date(2017, 1, 1), freq='H')[:-1])\
          .rename(columns={0: 'Time'})
    
Times['date'] = Times['Time'].dt.date
Times['hour'] = Times['Time'].dt.hour
Times['month'] = Times['Time'].dt.month
Times['weekday'] = Times['Time'].dt.weekday
Times = Times.drop('Time',1)

Times.tail()

,date,hour,month,weekday
23131,2016-12-31,19,12,5
23132,2016-12-31,20,12,5
23133,2016-12-31,21,12,5
23134,2016-12-31,22,12,5
23135,2016-12-31,23,12,5


### Merge AQI and Times dataframes together 

In [7]:
AQI = AQI.merge(Times, on=['date', 'hour'], how='right')
AQI = AQI.sort_values(by=['date','hour']).reset_index(drop=True)
print(len(AQI))

23136


### To make computations simpler, only take stations from 21 metro areas.
For each metro area, pick station with highest number of observations. See "stations" notebook for more details

In [68]:
metro_codes = pd.read_csv(path+'Station_Metro_Codes.csv')
metro_codes = metro_codes.drop('Unnamed: 0', 1)
stations_to_use = list(metro_codes['StationID'])
metro_codes

,StationID,Metro
0,1589A,Baotou
1,1012A,Beijing
2,1342A,Changsha
3,1434A,Chengdu
4,1419A,Chongqing
5,1282A,Fuzhou
6,1349A,Guangzhou
7,1129A,Harbin
8,1453A,Kunming
9,1456A,Lhasa


### Deal with missing observations: 

**Strategy: For any missing observation, take a weighted average of the most immediate (recorded) prior observation and most immediate (recorded) future observation.** The weights are based on how far away the recorded observations are from the missing observation. For example, let's say we are missing AQI data for 4:00am and 5:00am, but we know that AQI = 30 at 3:00am and AQI = 60 at 6:00am. Instead of simply plugging in the unweighted average AQI value (AQI = 45) for both hours, the code below will produce AQI = 40 for 4:00am and AQI = 50 for 5:00am. For the 4:00am missing value, twice as much weight is placed on the 3:00am recorded value than the 6:00am recorded value, resulting in a weighted average of (2/3 x 30)  + (1/3 x 60) = 40.  

In [9]:
def fill_missing_obs(DF, StationID):
    '''This function fills missing values for a particular station
    @StationID: string format 
    @DF: dataframe
    '''
    DF['missing_'+StationID] = np.where(DF[StationID]>-99, 0, 1)
    missing = DF[DF['missing_'+StationID]==1]
    
    for x in missing.index: #For each missing cell
        if x != 0 and x != len(DF)-1:
            non_missing_prior_index = x-1 #Find most recent past record 
            non_missing_post_index = x+1  
    
            while True:
                if non_missing_prior_index in missing.index:
                    non_missing_prior_index = non_missing_prior_index - 1
                else:
                    break
        
            while True:
                if non_missing_post_index in missing.index:
                    non_missing_post_index = non_missing_post_index + 1
                else:
                    break

            pre_AQI = DF.loc[non_missing_prior_index, StationID] 
            post_AQI = DF.loc[non_missing_post_index, StationID]

            priordiff = x - non_missing_prior_index
            postdiff = non_missing_post_index - x
            diff = priordiff + postdiff
            prior_weight = postdiff/diff
            post_weight = priordiff/diff
    
            DF.loc[x, StationID] = (prior_weight*pre_AQI) + (post_weight*post_AQI) 
    
    #else if x == 0 or x == len(DF)-1
    DF[StationID] = DF[StationID].fillna(method = 'ffill').fillna(method = 'bfill')
    return DF

In [10]:
def remove_columns(DF, list_of_stations):
    '''This function removes station data that will not be used in our model.
    @list_of_stations: a list of stations that we want to keep!
    '''
    cols = ['date', 'month', 'weekday', 'hour']
    stations = [x for x in list_of_stations]
    stations_missing = ['missing_'+y for y in list_of_stations]
    cols.extend(stations)
    cols.extend(stations_missing)
    new_DF = DF[cols]
    return new_DF

## Let's focus on predicting AQI at the 21 stations listed above 

In [11]:
stations_to_use = list(metro_codes['StationID'])

for s in stations_to_use:
    print(s)
    AQI = fill_missing_obs(AQI, s)

1589A
1012A
1342A
1434A
1419A
1282A
1349A
1129A
1453A
1456A
1152A
1406A
1310A
1144A
1099A
1493A
1330A
1465A
1482A
1488A
1316A


In [12]:
AQI_less = remove_columns(AQI, stations_to_use)
AQI_less.head()

,date,month,weekday,hour,1589A,1012A,1342A,1434A,1419A,1282A,...,missing_1406A,missing_1310A,missing_1144A,missing_1099A,missing_1493A,missing_1330A,missing_1465A,missing_1482A,missing_1488A,missing_1316A
0,2014-05-13,5,1,0.0,108.0,86.0,72.0,218.0,65.0,46.0,...,0,0,0,0,0,0,0,0,0,0
1,2014-05-13,5,1,1.0,117.0,84.0,80.0,230.0,63.0,58.0,...,0,0,0,0,0,0,0,0,0,0
2,2014-05-13,5,1,2.0,118.0,100.0,77.0,252.0,63.0,79.0,...,0,0,0,0,0,0,0,0,0,0
3,2014-05-13,5,1,3.0,116.0,95.0,68.0,283.0,69.0,42.0,...,0,0,0,0,0,0,0,0,0,0
4,2014-05-13,5,1,4.0,121.0,100.0,68.0,324.0,70.0,43.0,...,0,0,0,0,0,0,0,0,0,0


# Autoregressive Features 

For this task, we want to predict pollution levels far into the future (at least three days away). Therefore, in terms of autoregressive features, we will only use values that occur at least 24 hours in the past. 

In [13]:
def add_autoregressive_features(DF, StationID):
    time_steps = [24, 36, 48, 60, 72]
    for t in time_steps:
        lagged_values = np.empty(t)
        lagged_values[:] = np.nan
        lagged_values = list(lagged_values)
        lagged_values.extend(list(DF[StationID][:-t]))
        DF[StationID+' t-'+str(t)] = lagged_values
    return DF

In [14]:
datasets = []
for s in stations_to_use:
    datasets.append(add_autoregressive_features(AQI_less.copy(), s))
datasets[1].tail()

,date,month,weekday,hour,1589A,1012A,1342A,1434A,1419A,1282A,...,missing_1330A,missing_1465A,missing_1482A,missing_1488A,missing_1316A,1012A t-24,1012A t-36,1012A t-48,1012A t-60,1012A t-72
23131,2016-12-31,12,5,19.0,97.0,470.0,105.0,249.0,198.0,89.0,...,0,0,0,0,0,415.0,93.0,105.0,40.0,39.0
23132,2016-12-31,12,5,20.0,113.0,483.0,133.0,250.0,196.0,76.0,...,0,0,0,0,0,424.0,109.0,133.0,53.0,24.0
23133,2016-12-31,12,5,21.0,159.5,491.5,156.5,254.5,175.5,69.0,...,1,1,1,1,1,430.0,84.0,137.0,42.0,16.0
23134,2016-12-31,12,5,22.0,206.0,500.0,180.0,259.0,155.0,62.0,...,0,0,0,0,0,412.0,143.0,158.0,58.0,27.0
23135,2016-12-31,12,5,23.0,234.0,500.0,173.0,260.0,142.0,50.0,...,0,0,1,0,0,328.0,153.0,144.0,71.5,42.0


# Incorporate AQI values from other stations 

In [15]:
def add_AQI_other_stations(DF, StationID):
    time_steps = [24, 36, 48, 60, 72]
    for s in stations_to_use:
        if s == StationID:
            pass
        else:
            for t in time_steps:
                lagged_values = np.empty(t)
                lagged_values[:] = np.nan
                lagged_values = list(lagged_values)
                lagged_values.extend(list(DF[s][:-t]))
                DF[s+' t-'+str(t)] = lagged_values
    return DF   

In [16]:
for d in range(len(datasets)):
    df = datasets[d]
    datasets[d] = add_AQI_other_stations(df.copy(), stations_to_use[d])
datasets[1].tail()

,date,month,weekday,hour,1589A,1012A,1342A,1434A,1419A,1282A,...,1488A t-24,1488A t-36,1488A t-48,1488A t-60,1488A t-72,1316A t-24,1316A t-36,1316A t-48,1316A t-60,1316A t-72
23131,2016-12-31,12,5,19.0,97.0,470.0,105.0,249.0,198.0,89.0,...,162.0,123.0,82.0,80.0,74.0,247.0,259.0,276.0,305.0,233.0
23132,2016-12-31,12,5,20.0,113.0,483.0,133.0,250.0,196.0,76.0,...,135.0,132.0,107.0,88.0,73.0,246.0,270.0,294.0,309.0,244.0
23133,2016-12-31,12,5,21.0,159.5,491.5,156.5,254.5,175.5,69.0,...,165.0,130.0,124.0,107.0,78.0,261.0,273.0,287.0,330.0,230.0
23134,2016-12-31,12,5,22.0,206.0,500.0,180.0,259.0,155.0,62.0,...,170.0,142.0,105.0,62.0,78.0,266.0,284.0,304.0,332.0,227.0
23135,2016-12-31,12,5,23.0,234.0,500.0,173.0,260.0,142.0,50.0,...,178.0,135.0,108.0,57.5,114.0,276.0,306.0,293.0,332.5,231.0


In [18]:
def final_cleaning(DF, StationID):
    DF = DF.drop('date', 1)
    for s in stations_to_use:
        if s == StationID:
            DF = DF.drop(['missing_'+s], 1)
        else:
            DF = DF.drop([s, 'missing_'+s], 1)
    return DF

def turn_target_into_categorical_variable(DF, StationID):
    '''
    This function turns the target variable AQI into a categorical variable with 
    three classes.
    AQI 0-100: "Excellent to Good" -- class 0
    AQI 101-200: "Lightly to Moderately Polluted" -- class 1
    AQI 200+: "Heavily Polluted" -- class 2
    '''
    DF[StationID+'_target'] = np.where(DF[StationID]>200, 2, 1) 
    DF[StationID+'_target'] = np.where(DF[StationID]<101, 0, DF[StationID+'_target'])
    return DF

In [19]:
for d in range(len(datasets)):
    df = datasets[d]
    station = stations_to_use[d]
    print(station)
    datasets[d] = final_cleaning(df.copy(), station)
    datasets[d] = turn_target_into_categorical_variable(datasets[d], station)
datasets[1].tail()

1589A
1012A
1342A
1434A
1419A
1282A
1349A
1129A
1453A
1456A
1152A
1406A
1310A
1144A
1099A
1493A
1330A
1465A
1482A
1488A
1316A


,month,weekday,hour,1012A,1012A t-24,1012A t-36,1012A t-48,1012A t-60,1012A t-72,1589A t-24,...,1488A t-36,1488A t-48,1488A t-60,1488A t-72,1316A t-24,1316A t-36,1316A t-48,1316A t-60,1316A t-72,1012A_target
23131,12,5,19.0,470.0,415.0,93.0,105.0,40.0,39.0,99.0,...,123.0,82.0,80.0,74.0,247.0,259.0,276.0,305.0,233.0,2
23132,12,5,20.0,483.0,424.0,109.0,133.0,53.0,24.0,102.0,...,132.0,107.0,88.0,73.0,246.0,270.0,294.0,309.0,244.0,2
23133,12,5,21.0,491.5,430.0,84.0,137.0,42.0,16.0,105.0,...,130.0,124.0,107.0,78.0,261.0,273.0,287.0,330.0,230.0,2
23134,12,5,22.0,500.0,412.0,143.0,158.0,58.0,27.0,127.0,...,142.0,105.0,62.0,78.0,266.0,284.0,304.0,332.0,227.0,2
23135,12,5,23.0,500.0,328.0,153.0,144.0,71.5,42.0,135.0,...,135.0,108.0,57.5,114.0,276.0,306.0,293.0,332.5,231.0,2


In [24]:
TrainXs = []
TrainYs = []
TestXs = []
TestYs = []

for d in range(len(datasets)):
    SET = datasets[d]
    StationID = stations_to_use[d]
    Train = SET[:20000]
    Test = SET[20000:]

    #Drop training observations with insufficient data (e.g. records within the first 120 hours of dataset)
    Train = Train.dropna()
    TrainX = Train.drop([StationID, StationID+'_target'], 1)
    TrainY = Train[StationID+'_target']
    TestX = Test.drop([StationID, StationID+'_target'], 1)
    TestY = Test[StationID+'_target']
    
    TrainXs.append(TrainX)
    TrainYs.append(TrainY)
    TestXs.append(TestX)
    TestYs.append(TestY)

### Baselines under classification scheme 

In [25]:
datasets[0].groupby('1589A_target').count()[['month']]/len(datasets[0])

,month
1589A_target,
0,0.744338
1,0.218447
2,0.037215


In [57]:
def baselines_per_class(df, stationID):
    '''This function calculates the percentage of observations in each class (0, 1, and 2) for a 
    particular city. Returns a baseline for accuracy (class with maximum frequency) and a baseline 
    for recall (percentage of time Class=2 appears)
    '''
    #pick arbitrary column month
    acc_by_class = df.groupby(stationID+'_target').count()[['month']]/len(df)
    baseline_acc = round(acc_by_class['month'].max(), 3)
    baseline_rec = round(acc_by_class.loc[2, 'month'], 3)
    return baseline_acc, baseline_rec

In [58]:
baseline_accs = []
baseline_recs = []

for d in range(len(datasets)):
    baseline_acc, baseline_rec = baselines_per_class(datasets[d], stations_to_use[d])
    baseline_accs.append(baseline_acc)
    baseline_recs.append(baseline_rec)

# Initial Model -- Decision Tree Classifier

In [50]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics

DTs = []
predictions = []
actuals = []
recalls = []
accs = []

for d in range(len(datasets)):
    DT = DecisionTreeClassifier(min_samples_leaf=20)
    DT.fit(TrainXs[d], TrainYs[d])
    DTs.append(DT)
    preds = DT.predict(TestXs[d])
    predictions.append(preds)
    actual = TestYs[d]
    actuals.append(actual)
    score = DT.score(TestXs[d], TestYs[d])
    accs.append(round(score, 3))    
    try:
        recall_score = sklearn.metrics.recall_score(actual, preds, pos_label=2, average=None)[2]
        recalls.append(round(recall_score,3))
    except:
        recalls.append(0)

/Users/Brenton/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1045: UserWarning: Note that pos_label (set to 2) is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [62]:
AccuracyDF = pd.DataFrame(np.vstack((stations_to_use,metro_codes['Metro'],
                                     accs, baseline_accs, recalls, baseline_recs))).T

AccuracyDF = AccuracyDF.rename(columns={2: 'Accuracy', 3: 'Baseline Accuracy', 4: 'Recall',
                                       5: 'Baseline Recall'})

AccuracyDF = AccuracyDF.sort_values(by='Recall', ascending=False)
AccuracyDF

,0,1,Accuracy,Baseline Accuracy,Recall,Baseline Recall
12,1310A,Qingdao,0.782,0.823,0.484,0.026
15,1493A,Urumqi,0.67,0.682,0.424,0.119
1,1012A,Beijing,0.532,0.56,0.347,0.144
20,1316A,Zhengzhou,0.482,0.479,0.307,0.134
14,1099A,Shenyang,0.616,0.693,0.274,0.093
17,1465A,Xian,0.581,0.676,0.27,0.067
3,1434A,Chengdu,0.688,0.739,0.239,0.042
0,1589A,Baotou,0.675,0.744,0.21,0.037
18,1482A,Xining,0.686,0.75,0.156,0.03
9,1456A,Lhasa,0.767,0.899,0.123,0.012


### Feature Importances for Beijing
Pollution in Beijing at t-24 seems to be the most important feature. The second most important feature is pollution in Baotou at t-24 -- Baotou is a city about 400 miles west of Beijing. The third most important feature is pollution in Urumqi at t-60 -- Urumqi is in the far northwestern corner of China. Finally, month seems to be an important variable -- pollution in Beijing is higher during the winter. 

In [66]:
DFF = pd.DataFrame(np.vstack((TrainXs[1].columns.values, DTs[1].feature_importances_))).T
DFF.sort_values(by=1, ascending=False)

,0,1
3,1012A t-24,0.189243
8,1589A t-24,0.0654759
81,1493A t-60,0.0235145
0,month,0.021726
38,1129A t-24,0.0202423
88,1465A t-24,0.0155036
63,1310A t-24,0.0140296
57,1152A t-72,0.0134957
72,1144A t-72,0.0128893
4,1012A t-36,0.0125777
